In [7]:
from core.layers.gold import Gold
from core.layers.silver import Silver
from datetime import date
import pandas as pd
import pandasql as ps


class CnesEstabelecimentosMetrics(Gold):
    def __init__(self, year_month: str = "all"):
        super().__init__(name="cnes_estabelecimentos_metrics")
        self.year_month = year_month

        # entradas
        self.inputs = {
            "estabelecimentos": self.read_silver_parquet("cnes_estabelecimentos", year_month=self.year_month),
            "populacao": self._read_single_parquet(fs_client=self._gold_fs, path="populacao/data.parquet"),
        }

    def definition(self) -> pd.DataFrame:
        estab = self.inputs["estabelecimentos"].copy()
        pop = self.inputs["populacao"].copy()

        # ============================================================
        # filtros
        # ============================================================
        mask_sus = estab.get("TP_SUS_NAO_SUS", "").eq("S")
        mask_med = estab.get("DS_ATIVIDADE_PROFISSIONAL", "").astype(str).str.startswith("MEDICO", na=False)
        estab = estab[mask_sus & mask_med]

        keep_cols = [
            "CO_PROFISSIONAL_SUS",
            "NO_MUNICIPIO",
            "DS_ATIVIDADE_PROFISSIONAL",
            "TP_SUS_NAO_SUS",
            "CO_MUNICIPIO",
            "YYYYMM",
        ]
        estab = estab[[c for c in keep_cols if c in estab.columns]].copy()

        # tipos e chaves
        estab["CO_MUNICIPIO_SEM_DIGITO"] = pd.to_numeric(estab["CO_MUNICIPIO"], errors="coerce").astype("Int64")
        estab["YYYY"] = estab["YYYYMM"].astype(str).str[:4].astype("Int16")
        estab["MM"] = estab["YYYYMM"].astype(str).str[4:6]
        estab["MM"] = pd.Categorical(estab["MM"], categories=[f"{m:02d}" for m in range(1, 13)], ordered=True)

        # agrega profissionais únicos
        query = """
        SELECT
            CO_MUNICIPIO_SEM_DIGITO,
            NO_MUNICIPIO,
            DS_ATIVIDADE_PROFISSIONAL,
            TP_SUS_NAO_SUS,
            YYYY,
            MM,
            COUNT(DISTINCT CO_PROFISSIONAL_SUS) AS TOTAL_PROFISSIONAIS
        FROM estab
        WHERE TP_SUS_NAO_SUS = 'S'
        GROUP BY
            CO_MUNICIPIO_SEM_DIGITO,
            NO_MUNICIPIO,
            DS_ATIVIDADE_PROFISSIONAL,
            TP_SUS_NAO_SUS,
            YYYY,
            MM
        """
        g = ps.sqldf(query, locals())

        # normaliza população
        for c in ["CO_MUNICIPIO_SEM_DIGITO", "YYYY", "MM"]:
            if c not in pop.columns:
                raise KeyError(f"população: coluna obrigatória ausente: {c}")

        pop = pop.copy()
        pop["CO_MUNICIPIO_SEM_DIGITO"] = pd.to_numeric(pop["CO_MUNICIPIO_SEM_DIGITO"], errors="coerce").astype("Int64")
        pop["YYYY"] = pd.to_numeric(pop["YYYY"], errors="coerce").astype("Int16")
        pop["MM"] = pop["MM"].astype(str).str.zfill(2)

        # join e métrica
        join_keys = ["CO_MUNICIPIO_SEM_DIGITO", "YYYY", "MM"]
        cols_pop = join_keys + [
            "CO_UF", "NO_UF", "NO_REGIAO", "NO_MUNICIPIO_IBGE",
            "POPULACAO_MENSAL", "POPULACAO", "GROWTH_ABS", "GROWTH_PCT"
        ]
        cols_pop = [c for c in cols_pop if c in pop.columns]

        df = g.merge(pop[cols_pop], on=join_keys, how="left")
        df["PROFISSIONAIS_POR_1000"] = (
            (df["TOTAL_PROFISSIONAIS"] / df["POPULACAO_MENSAL"].replace({0: pd.NA})) * 1000
        )

        df["DATA_INGESTAO"] = pd.Timestamp.today().strftime("%Y-%m-%d")

        return df


# Teste
m = CnesEstabelecimentosMetrics(year_month="202105")
df = m.definition()
# saber porcentagem tem NO_MUNICIPIO_IBGE Nan
nan_count = df["NO_MUNICIPIO_IBGE"].isna().sum()
print(f"Porcentagem de NaN em NO_MUNICIPIO_IBGE: {nan_count / df.shape[0] * 100:.2f}%")

# mostrar NO_MUNICIPIO_IBGE distintos
df["NO_MUNICIPIO_IBGE"].dropna().unique()


Porcentagem de NaN em NO_MUNICIPIO_IBGE: 23.14%


array(['ALAMBARI', 'ALUMINIO', 'ARCO-IRIS', 'ASPASIA',
       'BOM SUCESSO DE ITARARE', 'BREJO ALEGRE', 'CAJATI', 'CANAS',
       'CANITAR', 'CRAVINHOS', 'CRISTAIS PAULISTA', 'CRUZALIA',
       'CRUZEIRO', 'CUBATAO', 'CUNHA', 'DESCALVADO', 'DIADEMA',
       'DIRCE REIS', 'DIVINOLANDIA', 'DOIS CORREGOS', 'DOLCINOPOLIS',
       'DOURADO', 'DRACENA', 'DUARTINA', 'DUMONT', 'ECHAPORA', 'ELDORADO',
       'ELIAS FAUSTO', 'ELISIARIO', 'EMBAUBA', 'EMBU DAS ARTES',
       'EMBU-GUACU', 'EMILIANOPOLIS', 'ENGENHEIRO COELHO',
       'ESPIRITO SANTO DO PINHAL', 'ESPIRITO SANTO DO TURVO',
       "ESTRELA D'OESTE", 'ESTRELA DO NORTE',
       'EUCLIDES DA CUNHA PAULISTA', 'FARTURA', 'FERNANDOPOLIS',
       'FERNANDO PRESTES', 'FERNAO', 'FERRAZ DE VASCONCELOS',
       'FLORA RICA', 'FLOREAL', 'FLORIDA PAULISTA', 'FLORINEA', 'FRANCA',
       'FRANCISCO MORATO', 'FRANCO DA ROCHA', 'GABRIEL MONTEIRO', 'GALIA',
       'GARCA', 'GASTAO VIDIGAL', 'GAVIAO PEIXOTO', 'GENERAL SALGADO',
       'GETULINA', 'GLICE

In [ ]:
import io
from core.layers import gold
from core.layers import silver

#read table from silver
def read_silver_table(table_name: str, year_month: str) -> pd.DataFrame:
    silver_fs = silver.fs
    file_path = f"{table_name}/{year_month}.parquet"
    file_client = silver_fs.get_file_client(file_path)

    download = file_client.download_file()
    downloaded_bytes = download.readall()

    # Carregar os bytes em um DataFrame do pandas
    df = pd.read_parquet(io.BytesIO(downloaded_bytes))
    return df

def read_gold_table(table_name: str) -> pd.DataFrame:
    gold_fs = gold
    file_path = f"{table_name}/data.parquet"
    file_client = gold_fs.get_file_client(file_path)

    download = file_client.download_file()
    downloaded_bytes = download.readall()

    # Carregar os bytes em um DataFrame do pandas
    df = pd.read_parquet(io.BytesIO(downloaded_bytes))
    return df

test = read_silver_table("cnes_estabelecimentos", "202201")
test

AttributeError: module 'core.layers.silver' has no attribute 'get_file_client'

In [10]:
import io
import pandas as pd
from core.infra.storage import gold  # 👈 importa a instância certa

def read_gold_parquet(table_name: str) -> pd.DataFrame:
    """
    Lê um Parquet da camada Gold usando a instância `gold` do Storage.
    Exemplo de caminho: gold/<table_name>/data.parquet
    """
    remote_path = f"{table_name}/202209.parquet"
    print(f"📥 Lendo gold/{remote_path} ...")

    # baixa bytes do arquivo
    file_client = gold.fs.get_file_client(remote_path)
    download = file_client.download_file()
    data = download.readall()

    # lê com pandas
    df = pd.read_parquet(io.BytesIO(data))
    print(f"✅ {len(df):,} linhas carregadas de {remote_path}")
    return df



df = read_gold_parquet("cnes_estabelecimentos_metrics")
df


📥 Lendo gold/cnes_estabelecimentos_metrics/202209.parquet ...
✅ 230,409 linhas carregadas de cnes_estabelecimentos_metrics/202209.parquet


,CO_MUNICIPIO_SEM_DIGITO,NO_MUNICIPIO,DS_ATIVIDADE_PROFISSIONAL,TP_SUS_NAO_SUS,YYYY,MM,TOTAL_PROFISSIONAIS,CO_UF,NO_UF,NO_REGIAO,NO_MUNICIPIO_IBGE,POPULACAO_MENSAL,POPULACAO,GROWTH_ABS,GROWTH_PCT,PROFISSIONAIS_POR_1000,DATA_INGESTAO
0,350010,ADAMANTINA,MEDICO ANATOMOPATOLOGISTA,S,2021,01,1,NaN,None,None,None,<NA>,NaN,<NA>,<NA>,<NA>,2025-11-03
1,350010,ADAMANTINA,MEDICO ANATOMOPATOLOGISTA,S,2021,02,1,NaN,None,None,None,<NA>,NaN,<NA>,<NA>,<NA>,2025-11-03
2,350010,ADAMANTINA,MEDICO ANATOMOPATOLOGISTA,S,2021,03,1,NaN,None,None,None,<NA>,NaN,<NA>,<NA>,<NA>,2025-11-03
3,350010,ADAMANTINA,MEDICO ANATOMOPATOLOGISTA,S,2021,04,1,NaN,None,None,None,<NA>,NaN,<NA>,<NA>,<NA>,2025-11-03
4,350010,ADAMANTINA,MEDICO ANATOMOPATOLOGISTA,S,2021,05,1,NaN,None,None,None,<NA>,NaN,<NA>,<NA>,<NA>,2025-11-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230404,355730,ESTIVA GERBI,MEDICO UROLOGISTA,S,2021,11,1,35.0,São Paulo,Campinas,ESTIVA GERBI,11524,11507.0,2,0.000174,0.086775,2025-11-03
230405,355730,ESTIVA GERBI,MEDICO UROLOGISTA,S,2021,12,1,35.0,São Paulo,Campinas,ESTIVA GERBI,11525,11507.0,1,0.000087,0.086768,2025-11-03
230406,355730,ESTIVA GERBI,MEDICO UROLOGISTA,S,2022,01,1,35.0,São Paulo,Campinas,ESTIVA GERBI,11527,11527.0,2,0.000174,0.086753,2025-11-03
230407,355730,ESTIVA GERBI,MEDICO UROLOGISTA,S,2022,02,1,35.0,São Paulo,Campinas,ESTIVA GERBI,11529,11527.0,2,0.000174,0.086738,2025-11-03
